In [18]:
import os
import codecs
from bs4 import BeautifulSoup
import re
from tqdm import tqdm

from helper.utils import flatten_json
import torch

In [2]:
_RESOURCE = '../resource'
_BOOK = os.path.join(_RESOURCE,'stroke_book.html')

In [3]:
def parse_html(file_html_path):
    f=codecs.open(file_html_path, 'r')
    html = f.read()
    soup = BeautifulSoup(html, features="html.parser")
    font_spans = [ data for data in soup.select('span') if 'font-size' in str(data) ]
    list_text = []
    list_size = []
    list_font = []

    for idx,element in tqdm(enumerate(font_spans),total = len(font_spans)):
        size = re.search(r'(?is)(font-size:)(.*?)(px)',str(element.get('style'))).group(2)

        font = str(re.search(r'(?is)(font-family:)(.*?)(;)',str(element.get('style'))).group(2))
        text = element.text

        list_size.append(size)
        list_font.append(font)
        list_text.append(text)
    
    return list_size,list_font,list_text

In [4]:
list_size,list_font,list_text = parse_html(_BOOK)

100%|██████████| 8939/8939 [00:00<00:00, 106619.28it/s]


In [5]:
def get_appendix(list_size,list_font,list_text):
    """
        get appendix
        format json
    """
    
    list_part = []
    list_track_part = []
    list_track_chapter = []
    
    dict_part = None
    list_chap = []
    list_section = None
    
    list_part_pad = []
    list_chap_pad = []
    
    memory_part = None
    dict_section = None
        
    record_flag = False
    dict_section = None
    
    count_part = 0
    count_chap = 0
    
    dict_chap = None
    
    memory_chap = None
    
    for idx, item in enumerate(list_text):
        
        ## catch Part
        
        if int(list_size[idx]) == 58:
            """
            fix bug TODO
            
            memory Part
            
            """

            part_title = item
    
            count_part += 1
            part_num = count_part

                
            if part_num != memory_part:

                ## switch to new part
                memory_part = part_num

                ## record old part
                if dict_part and dict_part not in list_part and dict_part['chapter']:
                    list_part.append(dict_part)

                ## init new part
                dict_part = {}
                dict_part['part_num'] = part_num
                dict_part['part_title'] = part_title

                ## idx to investigate
                dict_part['idx_part_text'] = idx
                
                ## idx for abstract
                dict_part['capacity'] = ['']
                
                dict_part['chapter'] = []

        ## catch Chapter
        if idx < len(list_text) - 1:
            
            if int(list_size[idx]) == 43 and list_text[idx-1].startswith('Chapter'): 

                list_track_chapter.append(item)

                ## init new chapter

                dict_chap = {}
                
                count_chap += 1
                dict_chap['chapter_num'] = count_chap
                dict_chap['chapter_title'] = item
                
                ## idx to investigate
                dict_chap['idx_chap_text'] = idx
                
                ## idx for abstract
                dict_chap['capacity'] = ['']
                
                dict_chap['section'] = []
                
                ## record chapter
                if dict_chap['chapter_title'] != memory_chap:
                    memory_chap = dict_chap['chapter_title']
                    
                    dict_part['chapter'].append(dict_chap)                
    
            if 'IScript' in list_font[idx] and int(list_size[idx]) > 20 and list_track_chapter and not item[0].isdigit() and not item.startswith('x') and not '_' in item and not '•' in item:
                if item.startswith('Attacking'):
                    record_flag = True


                if record_flag:
                    if int(list_size[idx]) == 28:
                                            
                        
                        if dict_section and dict_chap:

                            dict_chap['section'].append(dict_section)

                        
                        dict_section = {}
                        dict_section['section_title'] = item
                        ## idx to investigate
                        dict_section['idx_sec_text'] = idx
                        
                        ## idx for abstract
                        dict_section['capacity'] = ['']
                        
                        dict_section['subsection'] = []
                    elif int(list_size[idx]) == 23:
                        dict_subsection = {}
                        dict_subsection['subsection_title'] = item
                        dict_subsection['capacity'] = ['']
                        
                        ## idx to investigate
                        dict_subsection['idx_subsec_text'] = idx

                        dict_section['subsection'].append(dict_subsection)
        
        
        else:
            dict_chap['section'].append(dict_section)
            if dict_part not in list_part:
                list_part.append(dict_part) 
                
    ## fix bug ## last section of earlier chapter --> posterior chapter --> DONE
    
    for idx, part in enumerate(list_part):
    
        ## excute internal part
        internal_part = part['chapter']
        for idx_p, chap in enumerate(internal_part):
            if idx_p > 0:
                internal_part[idx_p-1]['section'].append(chap['section'].pop(0))

        ## excute external part
        if idx > 0:
            list_part[idx-1]['chapter'][-1]['section'].append(part['chapter'][0]['section'].pop(0))
                
    ## move In this part
    for idx, part in enumerate(list_part):
#         try:
#             print(part['chapter'][-1]['section'][-1]['subsection'][-1])
#         except:
#             print('Fail {}'.format(idx))
        if idx < len(list_part)-1:
            if part['chapter'][-1]['section'][-1]['subsection'][-1]:
#                 print('='*50)
#                 in_this_part = part['chapter'][-1]['section'][-1]['subsection'].[-1]
#                 list_part[idx+1]['abstract'] = part['chapter'][-1]['section'][-1]['subsection'].pop(-1)
                part['chapter'][-1]['section'][-1]['subsection'].pop(-1)
    return list_part

In [6]:
list_part = get_appendix(list_size,list_font,list_text)
# list_part

In [17]:
# list_part

In [8]:
def assign_idx(list_part):
    
    ## define idx distance
    
    
    ## flatten json
    list_idx = []
    count = 0
    part_flatten = flatten_json(list_part)
    for key in list(part_flatten.keys()):
        if 'idx' in key:
            list_idx.append(part_flatten[key])
        
        elif 'abs' in key or 'para' in key:
            count +=  1
    
#     print('='*50)
#     print(list_idx)
    ## distance between 2 items side by side
    list_dist = []
    
    for idx,d in enumerate(list_idx):
        if idx > 0:
            list_dist.append([list_idx[idx-1]+1,d-1])
            
#         elif idx == len(list_idx) - 1:
    list_dist.append([list_idx[-1]+1,7472])
    
#     print('='*50)
#     print(list_dist)
#     print(count,len(list_dist))
#     print(list_dist[-1])

    ## assign
    
    count_assign = 0
    
    for part in list_part:
        
#         print('count_assign',count_assign)
        
        part['capacity'] = list_dist[count_assign]
        count_assign += 1
        
        chapters = part['chapter']
        
        for chap in chapters:
            
#             print('count_assign',count_assign)
            chap['capacity'] = list_dist[count_assign]
            count_assign += 1
            
            sections = chap['section']
            
            for sec in sections:
                
#                 print('count_assign',count_assign)

                ## check subsection/abstract
                if sec['subsection']:
                    sec['capacity'] = list_dist[count_assign]
                    count_assign += 1
                    subsections = sec['subsection']
                
                    for subsec in subsections:

    #                     print('count_assign',count_assign)
    
                        subsec['capacity'] = list_dist[count_assign]
                        count_assign += 1
            
                else:
                    
                    sec['capacity'] = list_dist[count_assign]
                    count_assign += 1
                    
#                 count_assign += 1
                
                
    return list_part

In [9]:
book_complete = assign_idx(list_part)

In [16]:
# book_complete

In [13]:
file_structure = open(os.path.join(_RESOURCE,'structure.json'),'w')
for item in book_complete:
    string_item = str(item).replace(r"'",r'"')
    file_structure.write(string_item)
    file_structure.write('\n')

In [19]:
torch.save(list_text,os.path.join(_RESOURCE,'text.h5'))
torch.save(list_font,os.path.join(_RESOURCE,'font.h5'))
torch.save(list_size,os.path.join(_RESOURCE,'size.h5'))